In [120]:
import open3d as o3d
import numpy as np
import os
from tqdm import tqdm
import tensorflow as tf
import random

## Fix your path here

In [121]:
filepath_kri = ["/Users/kristofferkjaer/Desktop/mesh", "/Users/kristofferkjaer/Desktop/SPRING_MALE"]

filepath_karo = ["/Users/KAROLINE/Desktop/Fagprojekt/02466-Fagpakkeprojekt-Point-clouds/Data/Female","/Users/KAROLINE/Desktop/Fagprojekt/02466-Fagpakkeprojekt-Point-clouds/Data/Male"]

In [122]:
data_paths = filepath_kri

N = 5

pcds_females = []
pcds_males = []
pcds = [pcds_females, pcds_males]

for data in range(len(data_paths)):
    files = []
    folder_to_view = data_paths[data]
    list = pcds[data]
    for file in os.listdir(folder_to_view):
        files.append(file)
    for i in tqdm(range(N)):
        file = (folder_to_view + "/" + str(files[i]))
        mesh = o3d.io.read_triangle_mesh(file)
        pcd = mesh.sample_points_poisson_disk(2048)
        #pcd = pcd.voxel_down_sample(voxel_size=0.5)
        list.append(pcd)

100%|██████████| 5/5 [00:02<00:00,  1.97it/s]


In [123]:
# pcds_norm = [[], []]

# for points in range(len(pcds)): 
#     for i in tqdm(range(len(pcds_females))):
#         pcds_norm.append[points](np.asarray(pcds[points][i].normals))

### Augment

In [124]:
# pcds_females[2].estimate_normals(
#     search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
# o3d.visualization.draw_geometries([pcds_females[2]],
#                                   point_show_normal=True)

In [125]:
def augment(points):
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
    points = tf.random.shuffle(points)
    return points

### Convert to numpy array

In [126]:
pcds_females_arr = []
pcds_males_arr = []
pcds_arr = [pcds_females_arr, pcds_males_arr]


for points in range(len(pcds)): 
    for i in tqdm(range(len(pcds_females))):
        pcds_arr[points].append(np.asarray(pcds[points][i].points) )



100%|██████████| 5/5 [00:00<00:00, 12748.64it/s]


In [127]:
furthest_distance = 0

for points in range(len(pcds_arr)): 
    for i in range(len(pcds_males_arr)):
        distance = np.max(np.sqrt(np.sum(abs(pcds_arr[points][i])**2,axis=1)))
        if distance > furthest_distance:
            furthest_distance = distance
            
furthest_distance

1.0342500339088097

### Normalize (Update to normalize in regards to biggest person)

In [128]:
def normalize(points): # The input of this function is a point cloud object
    # This function normalizes a point cloud within a unit sphere
    points = np.asarray(points.points) 
    centroid = np.mean(points, axis=0)
    points -= centroid
    points / furthest_distance
    return points # The function return a numpy array of dimension nx3 where the points are normalized within a unit sphere

pcds_females_arr_norm = []
pcds_males_arr_norm = []
pcds_arr_norm = [pcds_females_arr_norm, pcds_males_arr_norm]


for points in range(len(pcds_arr)):
    for i in tqdm(range(len(pcds_females))):
        pcds_arr_norm[points].append(normalize(pcds[points][i]))

100%|██████████| 5/5 [00:00<00:00, 3188.61it/s]


In [129]:
furthest_distance = 0

for points in range(len(pcds_arr_norm)): 
    for i in range(len(pcds_males_arr)):
        distance = np.max(np.sqrt(np.sum(abs(pcds_arr_norm[points][i])**2,axis=1)))
        if distance > furthest_distance:
            furthest_distance = distance
            
furthest_distance

0.980824198175669

### To visualise point cloud

In [130]:
#o3d.visualization.draw_geometries([pcds_females[2]])